In [ ]:

2025-11-25 13:33:18 Tuesday first time create


参考 https://docs.pytorch.ac.cn/docs/stable/generated/torch.nn.Module.html，

借助 https://deepwiki.com/search

### get_submodule

In [2]:
import torch
import torch.nn as nn

class MyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Sequential(
                nn.Linear(20, 5)
            )
        )
        self.block2 = nn.Linear(5, 2)

model = MyNet()

# ----------------------------
# 获取子模块
# ----------------------------
m1 = model.get_submodule("block1.0")     # 第一个 Linear
m2 = model.get_submodule("block1.2.0")   # 嵌套 Sequential 中的 Linear
m3 = model.get_submodule("block2")       # 直接拿到 block2

m4 = model.get_submodule("")
print(m1)   # Linear(10 → 20)
print(m2)   # Linear(20 → 5)
print(m3)   # Linear(5 → 2)
print(m4)


Linear(in_features=10, out_features=20, bias=True)
Linear(in_features=20, out_features=5, bias=True)
Linear(in_features=5, out_features=2, bias=True)
MyNet(
  (block1): Sequential(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): ReLU()
    (2): Sequential(
      (0): Linear(in_features=20, out_features=5, bias=True)
    )
  )
  (block2): Linear(in_features=5, out_features=2, bias=True)
)


### PyTorch 的 `state_dict` 包含什么？

1. **`state_dict` 是模型可保存到 ckpt 的全部状态**，如：

   ```python
   torch.save(model.state_dict(), "model.pth")
   ```

2. **`state_dict` 只包含三类信息：**

   * **Parameters**：可训练参数（权重、偏置）。
   * **Buffers**：非训练状态（BN 均值方差、mask 等）。
   * **extra_state**：通过 `get_extra_state()` 明确返回的自定义状态。

除此之外，模型中的普通 Python 属性 **不会自动保存**。



具体而言，
### **类别 1：会保存进 state_dict 且会被 optimizer 优化（Parameters）**

1. 属于：

* `nn.Parameter`
* 模块里的可训练权重（权重 weight、偏置 bias）

2. 特点：

* 会自动进入 `state_dict()`
* 会被优化器更新（Adam、SGD）
* `.requires_grad = True`
* optimizer.state_dict() 中也会保存对应的动量等信息

3. 示例：

  ```python
  self.fc1 = nn.Linear(10, 20)
  ```

  进入 state_dict：

  ```
  fc1.weight
  fc1.bias
  ```

---

### **类别 2：保存进 state_dict 但不会被 optimizer 优化（Buffers）**

1.  属于：

* 使用 `register_buffer(name, tensor)` 添加的状态
* 例如：BN 中

  * running_mean
  * running_var

2.  特点：

* 会进入 `state_dict()`（自动）
* **不会有梯度**
* **不会被 optimizer 更新**
* 仍然会在 `.to(device)` 中迁移设备

3. 示例：

```python
self.register_buffer("running_scale", torch.tensor(1.0))
```

---

### **类别 3：保存到 state_dict["extra_state"] 的自定义状态（Extra State）**

1. 属于：

你自己在模型中定义的“其他信息”，通过：

```python
def get_extra_state(self):
    return {...}

def set_extra_state(self, state):
    ...
```

2. 特点：

* 会作为 **extra_state** 统一进入 `state_dict`
* 不属于参数，不属于 buffer
* 适用于 Python 对象、配置、统计量等

3. 示例：

```python
def get_extra_state(self):
    return {"scale": self.custom_scale}
```

保存后 state_dict 结构：

```python
{
  "fc1.weight": ...,
  "fc1.bias": ...,
  "running_scale": ...,
  "extra_state": {"scale": 999}
}
```


### ✅ **最终：PyTorch state_dict 的三大类别**

| 类别                    | 会进 state_dict        | 被 optimizer 更新 | 定义方式                      | 适用内容                  |
| --------------------- | -------------------- | -------------- | ------------------------- | --------------------- |
| **1. 参数（Parameters）** | ✔                    | ✔              | `nn.Parameter` / 层.weight | 权重、偏置                 |
| **2. Buffer**         | ✔                    | ❌              | `register_buffer`         | BN 统计量、mask、pos_embed |
| **3. Extra State**    | ✔（在 "extra_state" 中） | ❌              | `get_extra_state()`       | 自定义信息、Python 对象       |





